# Featuring engineering

In [1]:
class Feturing_engineering():
    
    #attributs
    file = ''
    def _init_(self,file):
        self.file = file
    
    
    # Méthode
    
     #Je supprime les colonnes avec 60% de valeurs manquantes
    def supprimer_valeur_manquante(self, file,num):
        column_with_nan = file.columns[file.isnull().any()]
        for column in column_with_nan:
             if file[column].isnull().sum()*100.0/file.shape[0] > num:
                     file.drop(column,1, inplace=True)
        return file
    
    def Imputation_valeur_manquante(self, file, num):
        #Je remplis les valeurs nan par la médiane de la colonnes
        column_with_nan = num.columns[num.isnull().any()]

        for col in column_with_nan:
            file[col].fillna(file[col].median(), inplace = True)
        return file
    
    def Imputation_valeur_manquante2(self, file,cat):
        #Je remplis les valeurs nan par le mode de la colonnes catégorielle
        column_with_nan = cat.columns[cat.isnull().any()]
 
        for col in column_with_nan:
            file[col].fillna(file[col].mode()[0], inplace = True)
        return file
    

    def convert_age(self,age_days_negative):
        age_days_positive = - age_days_negative
        age_years = age_days_positive/365
        return age_years
    
    def one_hot_encoder(self,file):
        cols = list(file.columns)
        file = pd.get_dummies(file)
        new_cols = [c for c in file.columns if c not in cols]
        print('%d columns were one hot encoded.' % len(new_cols))
        return file, new_cols
    
    def label_encoder(self,file):
        
        le = LabelEncoder()
        le_count = 0
        cols = []

        # Iterate through the columns
        for col in file:
            if col != "TARGET":
                if file[col].dtype == 'object':
                # If 2 or fewer unique categories
                    if len(list(file[col].unique())) <= 2 :
                        # Train on the training data
                        le.fit(file[col])
                        # Transform both training and testing data
                        file[col] = le.transform(file[col])
                        # Keep track of how many columns were label encoded
                        le_count += 1
                        cols.append(col)

        print('%d columns were label encoded.' % le_count)
        print('les colonnes sont : %s' % cols)
        return file
    
    def correlation_target(self, file):
        # map features to their absolute correlation values
        corr = file.corr().abs()
        # set equality (self correlation) as zero
        corr[corr == 1] = 0
        # of each feature, find the max correlation
        # and sort the resulting array in ascending order
        corr_cols = corr.max().sort_values(ascending=False)
        # display the highly correlated features
        display(corr_cols[corr_cols > 0.8])

        
    def display_importances(self, feature_importance_df_):
        cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:100].index
        best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
        plt.figure(figsize=(15, 20))
        sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
        plt.title('LightGBM Features (avg over folds)')
        plt.tight_layout()
        plt.savefig('lgbm_importances01.png')

    
    
    
    def correlation(self, file,num1,num2):
        #prendre les variable quantitative
        numerical = file.select_dtypes(include=[np.number]).columns
        #matrice de correlation
        corr = file[numerical].corr()
        #Prendre les index des features
        cols_high_corr = corr[((np.abs(corr) > 0.8) & (np.abs(corr) < 1)).any(1)].index
        corr_high_corr = file[cols_high_corr].corr()

        EDA().correlation_heatmap(corr_high_corr, num1,num2)
        
        
    def suppr_corr(self, file):
        #Identify Correlated Variables
        # Threshold for removing correlated variables
        threshold = 0.8

        # Absolute value correlation matrix
        corr_matrix = file.corr().abs()
        print("matrice de correlation head")
        corr_matrix.head()
        
        upper = corr_matrix.where(
        np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
        upper.head()
        # Select columns with correlations above threshold
        to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

        print('There are %d columns to remove.' % (len(to_drop)))
        print('#'*80)
        print(to_drop)
        file = file.drop(columns=to_drop)
        print('shape: ', file.shape)
        return to_drop, file
